In [1]:
import numpy as np
import pandas as pd
from interpret.glassbox import ExplainableBoostingClassifier
from libraries.feature_selection import *
from data_preparation.data_preparation import transform_data
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.feature_selection import SequentialFeatureSelector

In [2]:
x_train, y_train = transform_data(pd.read_csv("datasets/in_time.csv"))
x_oot, y_oot = transform_data(pd.read_csv("datasets/out_of_time.csv"))

In [16]:
model_for_performance = LogisticRegression()
model= ExplainableBoostingClassifier()

pipe = Pipeline([
    #('preprocessing' ,full_pipeline_logisitic),
    #('droping not used', FunctionTransformer(lambda x: x.drop(['scale__remainder__customer_id', 'remainder__remainder__application_date', 'remainder__application_status_transform__Application_status', 'remainder__remainder__Application data: employment date (main applicant)'], axis=1))),
    ('Delete features with gini lower than 0.01', GiniSelector(0.01)), 
    ('Smart Correlated Selection', SmartCorrelatedSelection( variables=None,
                                                             method="spearman",
                                                             threshold=0.8,
                                                             missing_values="raise",
                                                             selection_method="model_performance",
                                                             estimator=model_for_performance,).set_output(transform="pandas")),
    #('Cleaning semi-manualy overly correlated featuers',OverCorrelatedDropper(0.6)),
    #('Logistic regression', model)
], verbose = True)
pipe.fit(x_train, y_train)

[Pipeline]  (step 1 of 2) Processing Delete features with gini lower than 0.01, total=  14.0s
[Pipeline]  (step 2 of 2) Processing Smart Correlated Selection, total= 1.3min


Pipeline(steps=[('Delete features with gini lower than 0.01',
                 GiniSelector(threshold=0.01)),
                ('Smart Correlated Selection',
                 SmartCorrelatedSelection(estimator=LogisticRegression(),
                                          method='spearman',
                                          missing_values='raise',
                                          selection_method='model_performance'))],
         verbose=True)

In [5]:
x_transformed=pipe.transform(x_train)

In [13]:
corr=x_transformed.corr(method="spearman").values
for i in range(corr.shape[0]):
    for j in range(i):
        if(i!=j):
            if(corr[i,j]>=0.6):
                print(i,j)

16 15
18 17


In [17]:
model.fit(x_transformed,y_train)

ExplainableBoostingClassifier()

In [29]:
importances=[(i,j) for i,j in zip(x_transformed.columns ,model.term_importances())]
variables=[i[0] for i in sorted(importances, key=lambda x: x[1],reverse=True)[:9]]
variables

['utilized_limit_in_revolving_loans_H0',
 'DPD_term_loan_H2',
 'Default_flag_H1',
 'Default_flag_H2',
 'Default_flag_H3',
 'DPD_term_loan_H3',
 'Default_flag_H4',
 'DPD_term_loan_H6',
 'Default_flag_H5']

In [36]:
pd.concat([x_transformed[variables], y_train], axis=1).to_csv("variable.csv")

In [ ]:
""" 
('Sequential Feature Selector', SequentialFeatureSelector(model, 
                                                              n_features_to_select=9,
                                                              scoring='roc_auc', 
                                                              cv=5).set_output(transform="pandas")),

"""

In [3]:
train.describe()

,Customer_id,No_dependants,Time_in_address,Time_in_current_job,Credit_cards,Debit_cards,Active_accounts,Active_loans,Active_mortgages,Active_credit_card_lines,...,out_transactions_amt_H8,out_transactions_amt_H7,out_transactions_amt_H6,out_transactions_amt_H5,out_transactions_amt_H4,out_transactions_amt_H3,out_transactions_amt_H2,out_transactions_amt_H1,out_transactions_amt_H0,Target
count,3.100000e+05,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,...,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000,310000.000000
mean,3.499523e+07,1.159529,8.472011,4.733869,0.246939,1.520745,1.220213,1.040158,-8200.258065,-8808.576742,...,4875.162629,4873.649860,4875.012980,4874.377914,4872.908508,4874.697344,4873.802552,4874.401424,4874.218386,0.068506
std,2.886171e+06,0.959115,8.490954,5.152743,0.457813,0.728501,0.460074,0.196330,3840.832513,3238.394342,...,1628.568628,1627.367891,1627.495556,1629.180310,1625.356039,1627.949687,1627.401824,1628.402842,1627.676323,0.252613
min,3.000002e+07,0.000000,0.020000,0.030000,0.000000,1.000000,1.000000,1.000000,-9999.000000,-9999.000000,...,3112.430000,3108.890000,3108.690000,3110.620000,3104.110000,3113.580000,3104.130000,3100.980000,3102.910000,0.000000
25%,3.249342e+07,0.000000,2.690000,1.310000,0.000000,1.000000,1.000000,1.000000,-9999.000000,-9999.000000,...,3820.850000,3820.600000,3822.347500,3820.677500,3820.117500,3822.267500,3821.247500,3820.895000,3821.200000,0.000000
50%,3.499199e+07,1.000000,5.540000,2.970000,0.000000,1.000000,1.000000,1.000000,-9999.000000,-9999.000000,...,4351.500000,4348.850000,4350.695000,4349.075000,4350.165000,4350.735000,4348.365000,4348.950000,4349.800000,0.000000
75%,3.749224e+07,2.000000,11.160000,6.230000,0.000000,2.000000,1.000000,1.000000,-9999.000000,-9999.000000,...,5298.022500,5297.187500,5300.130000,5294.427500,5299.130000,5294.782500,5300.202500,5298.062500,5298.127500,0.000000
max,3.999994e+07,4.000000,67.970000,49.410000,2.000000,4.000000,3.000000,2.000000,1.000000,2.000000,...,20353.100000,22575.640000,19739.770000,22748.870000,20216.120000,19710.560000,20713.020000,19914.480000,20969.010000,1.000000
